## Libs

In [1]:
import matplotlib as matpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

# Using the Tensorflow backend (default).
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from tensorflow import set_random_seed
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping

# advanced plotting
import seaborn as sns
plt.style.use('seaborn-darkgrid')
%matplotlib inline

Using TensorFlow backend.


## Data

In [11]:
%store -r data_TIP

In [12]:
mm = MinMaxScaler(feature_range = (0,1))
dataset = mm.fit_transform(data_TIP)

In [13]:
split = 0.6

train_size = int(len(dataset) * split)
#validation
test_size = len(dataset) - train_size

train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

print("training, test set: " + str((len(train), len(test))))

training, test set: (313, 209)


In [14]:
def input_dataset(dataset, window):
    data_X, data_y = [], []
    for i in range(len(dataset) - window - 1):
        a = dataset[i:(i + window), 0]
        data_X.append(a)
        data_y.append(dataset[i + window, 0])
    return(np.array(data_X), np.array(data_y))


In [15]:
# New testing and training sets for rolling forecast.
window = 1
train_X, train_Y = input_dataset(train, window)
test_X, test_Y = input_dataset(test, window)
print("Original train shape:")
print(train_X.shape)

# Reshape input data to match Keras format.
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("New train shape:")
print(train_X.shape)

Original train shape:
(311, 1)
New train shape:
(311, 1, 1)


## 1 Epoch

In [7]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = 'mean_squared_error', 
                  optimizer = 'adam')
    earlyStop=EarlyStopping(monitor='val_loss',verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 1, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

W0919 06:57:51.729141 4444517824 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0919 06:57:51.731282 4444517824 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0919 06:57:51.734215 4444517824 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0919 06:57:51.959274 4444517824 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0919 06:57:52.235514 4444517824 deprecation.py:323] From //anaconda3/li

Epoch 1/1
311/311 [==============================] - 5s 17ms/step - loss: 0.0016


In [9]:
def prediction_score(model, X, Y):
    # Make predictions using input data
    pred = mm.inverse_transform(model.predict(X))
    # Show Y on original scale
    original_data = mm.inverse_transform([Y])
    # RMSE.
    score = math.sqrt(mean_squared_error(original_data[0], pred[:, 0]))
    return(score, pred)

In [9]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 1.75 RMSE


## 1000 Epochs

In [10]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 1000, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/1000
311/311 [==============================] - 5s 18ms/step - loss: 0.0014
Epoch 2/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0052
Epoch 3/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0029
Epoch 4/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0017
Epoch 5/1000
311/311 [==============================] - 3s 9ms/step - loss: 0.0012
Epoch 6/1000
311/311 [==============================] - 3s 9ms/step - loss: 9.3094e-04
Epoch 7/1000
311/311 [==============================] - 3s 8ms/step - loss: 8.2190e-04
Epoch 8/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.7456e-04
Epoch 9/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.5444e-04
Epoch 10/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.4646e-04
Epoch 11/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.4376e-04
Epoch 12/1000
311/311 [==============================] - 3s 

311/311 [==============================] - 3s 9ms/step - loss: 7.1482e-04
Epoch 187/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.0950e-04
Epoch 188/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.0388e-04
Epoch 189/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.9866e-04
Epoch 190/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.9333e-04
Epoch 191/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.8839e-04
Epoch 192/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.8439e-04
Epoch 193/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.8104e-04
Epoch 194/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.7871e-04
Epoch 195/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.7705e-04
Epoch 196/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.7605e-04
Epoch 197/1000
311/311 [============

311/311 [==============================] - 3s 9ms/step - loss: 7.7804e-04
Epoch 370/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.7836e-04
Epoch 371/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.7781e-04
Epoch 372/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.7808e-04
Epoch 373/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.7873e-04
Epoch 374/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.7936e-04
Epoch 375/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.7893e-04
Epoch 376/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.7794e-04
Epoch 377/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.7734e-04
Epoch 378/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.7783e-04
Epoch 379/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.7830e-04
Epoch 380/1000
311/311 [============

311/311 [==============================] - 3s 8ms/step - loss: 7.6740e-04
Epoch 462/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.6621e-04
Epoch 463/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.6629e-04
Epoch 464/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.6536e-04
Epoch 465/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.6465e-04
Epoch 466/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.6447e-04
Epoch 467/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.6314e-04
Epoch 468/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.6264e-04
Epoch 469/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.6246e-04
Epoch 470/1000
311/311 [==============================] - 3s 8ms/step - loss: 7.6122e-04
Epoch 471/1000
311/311 [==============================] - 3s 9ms/step - loss: 7.6048e-04
Epoch 472/1000
311/311 [============

311/311 [==============================] - 3s 9ms/step - loss: 6.9157e-04
Epoch 646/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.9149e-04
Epoch 647/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.9165e-04
Epoch 648/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.9128e-04
Epoch 649/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.9049e-04
Epoch 650/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.8935e-04
Epoch 651/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.8868e-04
Epoch 652/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.8909e-04
Epoch 653/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.8975e-04
Epoch 654/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.9025e-04
Epoch 655/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.8996e-04
Epoch 656/1000
311/311 [============

311/311 [==============================] - 3s 8ms/step - loss: 6.6799e-04
Epoch 738/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.6815e-04
Epoch 739/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.6827e-04
Epoch 740/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.6809e-04
Epoch 741/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.6868e-04
Epoch 742/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.6820e-04
Epoch 743/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.6783e-04
Epoch 744/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.6739e-04
Epoch 745/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.6721e-04
Epoch 746/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.6707e-04
Epoch 747/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.6727e-04
Epoch 748/1000
311/311 [============

311/311 [==============================] - 3s 8ms/step - loss: 6.4174e-04
Epoch 922/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.4066e-04
Epoch 923/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.4066e-04
Epoch 924/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.4066e-04
Epoch 925/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.4077e-04
Epoch 926/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.4099e-04
Epoch 927/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.4088e-04
Epoch 928/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.4144e-04
Epoch 929/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.4103e-04
Epoch 930/1000
311/311 [==============================] - 3s 9ms/step - loss: 6.4088e-04
Epoch 931/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.3990e-04
Epoch 932/1000
311/311 [============

In [11]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)

print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 0.99 RMSE


## 2000 Epochs

In [7]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 2000, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

W0919 13:38:32.661674 4526294464 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0919 13:38:32.663697 4526294464 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0919 13:38:32.666371 4526294464 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0919 13:38:32.835674 4526294464 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0919 13:38:33.049219 4526294464 deprecation.py:323] From //anaconda3/li

Epoch 1/2000
311/311 [==============================] - 3s 11ms/step - loss: 0.0015
Epoch 2/2000
311/311 [==============================] - 2s 5ms/step - loss: 0.0054
Epoch 3/2000
311/311 [==============================] - 2s 6ms/step - loss: 0.0030
Epoch 4/2000
311/311 [==============================] - 2s 6ms/step - loss: 0.0018
Epoch 5/2000
311/311 [==============================] - 2s 6ms/step - loss: 0.0012
Epoch 6/2000
311/311 [==============================] - 2s 6ms/step - loss: 9.4705e-04
Epoch 7/2000
311/311 [==============================] - 2s 6ms/step - loss: 8.2866e-04
Epoch 8/2000
311/311 [==============================] - 2s 6ms/step - loss: 7.7718e-04
Epoch 9/2000
311/311 [==============================] - 2s 6ms/step - loss: 7.5521e-04
Epoch 10/2000
311/311 [==============================] - 2s 6ms/step - loss: 7.4639e-04
Epoch 11/2000
311/311 [==============================] - 2s 6ms/step - loss: 7.4335e-04
Epoch 12/2000
311/311 [==============================] - 2s 

311/311 [==============================] - 2s 8ms/step - loss: 6.9446e-04
Epoch 95/2000
311/311 [==============================] - 2s 8ms/step - loss: 6.9630e-04
Epoch 96/2000
311/311 [==============================] - 2s 8ms/step - loss: 6.9784e-04
Epoch 97/2000
311/311 [==============================] - 2s 8ms/step - loss: 6.9958e-04
Epoch 98/2000
311/311 [==============================] - 2s 8ms/step - loss: 7.0113e-04
Epoch 99/2000
311/311 [==============================] - 2s 8ms/step - loss: 7.0268e-04
Epoch 100/2000
311/311 [==============================] - 2s 8ms/step - loss: 7.0467e-04
Epoch 101/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.0626e-04
Epoch 102/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.0802e-04
Epoch 103/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.0975e-04
Epoch 104/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.1097e-04
Epoch 105/2000
311/311 [=================

311/311 [==============================] - 2s 7ms/step - loss: 7.0666e-04
Epoch 187/2000
311/311 [==============================] - 2s 8ms/step - loss: 7.0203e-04
Epoch 188/2000
311/311 [==============================] - 2s 7ms/step - loss: 6.9725e-04
Epoch 189/2000
311/311 [==============================] - 2s 7ms/step - loss: 6.9257e-04
Epoch 190/2000
311/311 [==============================] - 2s 7ms/step - loss: 6.8771e-04
Epoch 191/2000
311/311 [==============================] - 2s 7ms/step - loss: 6.8359e-04
Epoch 192/2000
311/311 [==============================] - 2s 7ms/step - loss: 6.7962e-04
Epoch 193/2000
311/311 [==============================] - 2s 7ms/step - loss: 6.7639e-04
Epoch 194/2000
311/311 [==============================] - 2s 7ms/step - loss: 6.7388e-04
Epoch 195/2000
311/311 [==============================] - 2s 7ms/step - loss: 6.7216e-04
Epoch 196/2000
311/311 [==============================] - 2s 7ms/step - loss: 6.7112e-04
Epoch 197/2000
311/311 [============

311/311 [==============================] - 2s 8ms/step - loss: 7.8300e-04
Epoch 279/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.8377e-04
Epoch 280/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.8435e-04
Epoch 281/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.8497e-04
Epoch 282/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.8562e-04
Epoch 283/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.8611e-04
Epoch 284/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.8667e-04
Epoch 285/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.8708e-04
Epoch 286/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.8744e-04
Epoch 287/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.8790e-04
Epoch 288/2000
311/311 [==============================] - 2s 7ms/step - loss: 7.8840e-04
Epoch 289/2000
311/311 [============

...

311/311 [==============================] - 2s 6ms/step - loss: 6.1619e-04
Epoch 1560/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1554e-04
Epoch 1561/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1582e-04
Epoch 1562/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1574e-04
Epoch 1563/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1583e-04
Epoch 1564/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1587e-04
Epoch 1565/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1532e-04
Epoch 1566/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1557e-04
Epoch 1567/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1522e-04
Epoch 1568/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1644e-04
Epoch 1569/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1514e-04
Epoch 1570/2000
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 6.1406e-04
Epoch 1651/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1433e-04
Epoch 1652/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1408e-04
Epoch 1653/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1387e-04
Epoch 1654/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1431e-04
Epoch 1655/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1400e-04
Epoch 1656/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1389e-04
Epoch 1657/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1400e-04
Epoch 1658/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1450e-04
Epoch 1659/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1394e-04
Epoch 1660/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1372e-04
Epoch 1661/2000
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 6.1254e-04
Epoch 1742/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1216e-04
Epoch 1743/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1297e-04
Epoch 1744/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1262e-04
Epoch 1745/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1226e-04
Epoch 1746/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1233e-04
Epoch 1747/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1273e-04
Epoch 1748/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1259e-04
Epoch 1749/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1197e-04
Epoch 1750/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1273e-04
Epoch 1751/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1269e-04
Epoch 1752/2000
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 6.1140e-04
Epoch 1833/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1173e-04
Epoch 1834/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1103e-04
Epoch 1835/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1091e-04
Epoch 1836/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1123e-04
Epoch 1837/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1147e-04
Epoch 1838/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1123e-04
Epoch 1839/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1086e-04
Epoch 1840/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1109e-04
Epoch 1841/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1120e-04
Epoch 1842/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1142e-04
Epoch 1843/2000
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 6.1008e-04
Epoch 1924/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.0980e-04
Epoch 1925/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.0947e-04
Epoch 1926/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.0934e-04
Epoch 1927/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.0988e-04
Epoch 1928/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.0997e-04
Epoch 1929/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.1001e-04
Epoch 1930/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.0927e-04
Epoch 1931/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.0941e-04
Epoch 1932/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.0955e-04
Epoch 1933/2000
311/311 [==============================] - 2s 6ms/step - loss: 6.0981e-04
Epoch 1934/2000
311/311 [=

In [10]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 0.88 RMSE


## 2500 Epochs

In [16]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 2500, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/2500
311/311 [==============================] - 4s 12ms/step - loss: 0.0015
Epoch 2/2500
311/311 [==============================] - 2s 5ms/step - loss: 0.0056
Epoch 3/2500
311/311 [==============================] - 2s 5ms/step - loss: 0.0031
Epoch 4/2500
311/311 [==============================] - 2s 5ms/step - loss: 0.0018
Epoch 5/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0012
Epoch 6/2500
311/311 [==============================] - 2s 5ms/step - loss: 9.4981e-04
Epoch 7/2500
311/311 [==============================] - 2s 5ms/step - loss: 8.3068e-04
Epoch 8/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7918e-04
Epoch 9/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.5740e-04
Epoch 10/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.4887e-04
Epoch 11/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.4616e-04
Epoch 12/2500
311/311 [==============================] - 2s 

311/311 [==============================] - 2s 6ms/step - loss: 7.0243e-04
Epoch 95/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.0434e-04
Epoch 96/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.0609e-04
Epoch 97/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.0783e-04
Epoch 98/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.0972e-04
Epoch 99/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.1159e-04
Epoch 100/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.1347e-04
Epoch 101/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.1521e-04
Epoch 102/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.1677e-04
Epoch 103/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.1851e-04
Epoch 104/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.2043e-04
Epoch 105/2500
311/311 [=================

311/311 [==============================] - 2s 6ms/step - loss: 7.1234e-04
Epoch 187/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.0653e-04
Epoch 188/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.0066e-04
Epoch 189/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.9505e-04
Epoch 190/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.9014e-04
Epoch 191/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.8603e-04
Epoch 192/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.8264e-04
Epoch 193/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.8019e-04
Epoch 194/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.7829e-04
Epoch 195/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.7704e-04
Epoch 196/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.7647e-04
Epoch 197/2500
311/311 [============

311/311 [==============================] - 2s 6ms/step - loss: 7.8703e-04
Epoch 279/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.8743e-04
Epoch 280/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.8783e-04
Epoch 281/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.8820e-04
Epoch 282/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.8863e-04
Epoch 283/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.8933e-04
Epoch 284/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.8921e-04
Epoch 285/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.8965e-04
Epoch 286/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.8988e-04
Epoch 287/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.9007e-04
Epoch 288/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.9026e-04
Epoch 289/2500
311/311 [============

311/311 [==============================] - 2s 6ms/step - loss: 7.7374e-04
Epoch 371/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7264e-04
Epoch 372/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7174e-04
Epoch 373/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7142e-04
Epoch 374/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7152e-04
Epoch 375/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7130e-04
Epoch 376/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7124e-04
Epoch 377/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7101e-04
Epoch 378/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7122e-04
Epoch 379/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7120e-04
Epoch 380/2500
311/311 [==============================] - 2s 6ms/step - loss: 7.7148e-04
Epoch 381/2500
311/311 [============

...

311/311 [==============================] - 2s 6ms/step - loss: 6.0796e-04
Epoch 2107/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0703e-04
Epoch 2108/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0753e-04
Epoch 2109/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0759e-04
Epoch 2110/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0748e-04
Epoch 2111/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0755e-04
Epoch 2112/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0691e-04
Epoch 2113/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0724e-04
Epoch 2114/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0742e-04
Epoch 2115/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0731e-04
Epoch 2116/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0706e-04
Epoch 2117/2500
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 6.0625e-04
Epoch 2198/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0542e-04
Epoch 2199/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0676e-04
Epoch 2200/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0565e-04
Epoch 2201/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0531e-04
Epoch 2202/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0642e-04
Epoch 2203/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0582e-04
Epoch 2204/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0584e-04
Epoch 2205/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0617e-04
Epoch 2206/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0570e-04
Epoch 2207/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0579e-04
Epoch 2208/2500
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 6.0473e-04
Epoch 2289/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0444e-04
Epoch 2290/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0425e-04
Epoch 2291/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0495e-04
Epoch 2292/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0436e-04
Epoch 2293/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0454e-04
Epoch 2294/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0515e-04
Epoch 2295/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0435e-04
Epoch 2296/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0520e-04
Epoch 2297/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0484e-04
Epoch 2298/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0411e-04
Epoch 2299/2500
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 6.0281e-04
Epoch 2380/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0245e-04
Epoch 2381/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0253e-04
Epoch 2382/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0293e-04
Epoch 2383/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0249e-04
Epoch 2384/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0294e-04
Epoch 2385/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0355e-04
Epoch 2386/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0296e-04
Epoch 2387/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0257e-04
Epoch 2388/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0344e-04
Epoch 2389/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0327e-04
Epoch 2390/2500
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 6.0232e-04
Epoch 2471/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0204e-04
Epoch 2472/2500
311/311 [==============================] - 2s 7ms/step - loss: 6.0205e-04
Epoch 2473/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0170e-04
Epoch 2474/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0232e-04
Epoch 2475/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0184e-04
Epoch 2476/2500
311/311 [==============================] - 2s 6ms/step - loss: 6.0175e-04
Epoch 2477/2500
311/311 [==============================] - 2s 7ms/step - loss: 6.0245e-04
Epoch 2478/2500
311/311 [==============================] - 2s 7ms/step - loss: 6.0189e-04
Epoch 2479/2500
311/311 [==============================] - 2s 7ms/step - loss: 6.0155e-04
Epoch 2480/2500
311/311 [==============================] - 2s 7ms/step - loss: 6.0178e-04
Epoch 2481/2500
311/311 [=

In [17]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 0.86 RMSE
